In [1]:
import pandas as pd
pd.set_option("display.max_columns", 999)
pd.set_option("display.max_row", 999)
import geopandas as gpd
import numpy as np
import requests
import math
import sys
import graphviz 
import csv
import sqlite3
import re
import codecs
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn import tree as tr
import os; os.getcwd()

'/Users/belenmichel/Desktop/MSCAPP/7_Machine Learning/4_FinalProject_Crimes/ml-nc-recidivism/source'

In [7]:
# Constants
data_dir = "../ncdoc_data/data"

In [3]:
# # Pre-ProcessData

def to_date(df, attribute_lst, years_range=[1800, 2100]):
    '''
    Converts the data type of a string in the format YYYY-MM-DD to a datetime  
    '''
    df = df.apply(out_of_range_to_none, axis=1, args=(years_range, attribute_lst))
    for var in attribute_lst:
        df[var] = df[var].astype('datetime64[s]')#, errors = 'ignore')
    return df


def out_of_range_to_none(row, year_range, column_lst): 
    '''
    Converts a str representing a date out of the intended range to None
    '''
    for col in column_lst:
        year = int(row[col].split("-")[0])
        if year < year_range[0] or year > year_range[1]:
            row[col] = None 
    return row




def to_int(df, attribute_lst):
    '''
    Converts the data type of a string in the format YYYY-MM-DD to a datetime  
    '''
    for var in attribute_lst:
        df[var] = pd.to_numeric(df[var], errors='coerce', downcast='integer') 
    return df

def remove_outliers(df, attribute_lst, sd_threshold=3):
    '''
    Takes a dataframe and number of standard deviations to be considered 
    as outlier and returns a df without the observation that have one or
    or more outliers in it's attributes
    input:
        df: pandas data frame
        sd_threshold: standard deviations
    output:
        the new datafrane without outliers
    '''   
    
    return(df[(np.abs(stats.zscore(df[attribute_lst])) < sd_threshold).all(axis=1)])


def fill_nan(df, attributes_lst):
    '''
    Fills the nan with the mean
    input:
        df: pandas data frame
        attributes_lst: list of attributes names
    output:
        dataframe with the replaced nan
    '''   
    for attribute in attributes_lst: 
        df[attribute].fillna(df[attribute].mean(), inplace=True)

def clean_str(col_names):
    '''
    Removes special characters from a string. 
    input:
        col_names: string with column names
    output:
        sting where special characters where removed
    ''' 
    #TODO: replace this ad-hoc function with a re function. 
    #col_names = re.sub(pattern, repl, string, count=0, flags=0) 
    col_names = col_names.replace('.','')
    col_names = col_names.replace('#','')
    col_names = col_names.replace('-','')
    col_names = col_names.replace('(','')
    col_names = col_names.replace(')','')
    col_names = col_names.replace('&','')
    col_names = col_names.replace('/','')
    col_names = col_names.replace('__','')
    return col_names


# # Generate Features/ Predictors


def discretize_variable(df, attribute_lst):
    '''
    Converts continuous variables into discrete variables
    input:
        df: pandas data frame
        attributes_lst: list of attributes names
    output:
        dataframe with the new variables
    ''' 

    for var in attribute_lst:
        new_var = var + 'cat'
        df[new_var] = pd.qcut(df[var], 10, duplicates="drop", labels=False)
    return df

def categorical_to_dummy(df, attribute_lst):
    '''
    Converts categorical variables into one variabel dummies for each category. 
    input:
        df: pandas data frame
        attributes_lst: list of attributes names
    output:
        dataframe with the new variables
    ''' 

    for var in attribute_lst:
        df = pd.get_dummies(df, columns=[var])
    return df

def remove_attribute(df, attribute_lst):
    '''
    Removes attributes in the list from the data frame
    '''     
    return df.drop(attribute_lst, axis=1)

def keep_attribute(df, attribute_lst):
    '''
    Keeps attributes in the list in the data frame
    '''
    df = df.loc[:, attribute_lst]
    return df

def flag_to_dummy(df, attribute_lst, rename=True):
    '''
    Converts a flag variable to a dummy with 1 for Yes and 0 for No
    '''
    for var in attribute_lst:
        df[var] = df[var].map({'Y': 1, 'N': 0, 'Yes': 1, 'No': 0, 'T': 1, 'F': 0,\
                               'True': 1, 'False': 0, 'OPEN': 1, 'CLOSED': 0})
        if rename:
            new_var_name = var[:-5]
            df.rename(index=str, columns={var: new_var_name}, inplace=True)
    return df

def gender_to_dummy(df, gender_var):  
    '''
    Converts a gender indicative variable to a dummy with 1 for female and 0 for male
    '''
    df[gender_var] = df[gender_var].map({'FEMALE': 1, 'MALE': 0, 'F': 1, 'M': 0})
    df.rename(index=str, columns={gender_var: "FEMALE"}, inplace=True)
    return df




In [282]:
#Cleaning Inmate Profile

file_name = os.path.join(data_dir, "preprocessed/INMT4AA1.csv")
Inmate_Profile = pd.read_csv(file_name) #, nrows=400
Inmate_Profile = Inmate_Profile[Inmate_Profile["INMATE_IS_FELON/MISDEMEANANT"]=="FELON"]
Inmate_Profile = flag_to_dummy(Inmate_Profile, ['ESCAPE_HISTORY_FLAG', 'PRIOR_INCARCERATIONS_FLAG'])
Inmate_Profile = gender_to_dummy(Inmate_Profile, 'INMATE_GENDER_CODE')
Inmate_Profile = categorical_to_dummy(Inmate_Profile, ['INMATE_RACE_CODE'])
#Remove outliers for TOTAL_SENTENCE_LENGTHIN_DAYS, LENGTH_OF_CURRENT_INCARCERATN, LENGTH_OF_RULING_SENTENCES
#Inmate_Profile = remove_outliers(Inmate_Profile, sd_threshold=3)
Inmate_Profile = to_date(Inmate_Profile, ["INMATE_BIRTH_DATE","INMATE_ADMISSION_DATE","FINAL_RULING_PED",\
                              "FINAL_RULING_TRD", "FINAL_RULING_PRD", "FINAL_RULING_MAX_RELEASE_DATE",\
                              "DATE_TRD_&_PRD_LAST_COMPUTED", "DATE_OF_LAST_ARREST_ON_PAROLE"])
Inmate_Profile["AGE_IN_DAYS"] = Inmate_Profile["INMATE_ADMISSION_DATE"] - Inmate_Profile["INMATE_BIRTH_DATE"]
Inmate_Profile = keep_attribute(Inmate_Profile, ['INMATE_DOC_NUMBER','INMATE_LAST_NAME','INMATE_FIRST_NAME',\
                                'FEMALE',"INMATE_RACE_CODE_BLACK", "INMATE_RACE_CODE_INDIAN",\
                                "INMATE_RACE_CODE_OTHER", "INMATE_RACE_CODE_WHITE","AGE_IN_DAYS"\
                                'ESCAPE_HISTORY', 'PRIOR_INCARCERATIONS',\
                                "INMATE_BIRTH_DATE","INMATE_ADMISSION_DATE","FINAL_RULING_PED",\
                                "FINAL_RULING_TRD", "FINAL_RULING_PRD", "FINAL_RULING_MAX_RELEASE_DATE",\
                                "DATE_TRD_&_PRD_LAST_COMPUTED", "DATE_OF_LAST_ARREST_ON_PAROLE",\
                                "TOTAL_SENTENCE_LENGTHIN_DAYS", "LENGTH_OF_CURRENT_INCARCERATN",\
                                "LENGTH_OF_RULING_SENTENCES"])

new_file_name = file_name[:-4] + '_cleaned.csv'
Inmate_Profile.to_csv(new_file_name)

#Shall we include?:
#TYPE_OF_LAST_INMATE_MOVEMENT
#MOST_SERIOUS_OFFNSE_CURR_INCAR

#CURRENT_COMMITMENT_PREFIX
#CURRENT_SENTENCE_COMPONENT

#LAST_RULING_PRD_COMMITMENT
#LAST_RULING_PRD_COMPONENT

In [ ]:
#Cleaning SentenceComputation

file_name = os.path.join(data_dir, "preprocessed/INMT4BB1.csv")
Sentence_Computation = pd.read_csv(file_name, low_memory=False)
Sentence_Computation = categorical_to_dummy(Sentence_Computation, ['INMATE_COMPUTATION_STATUS_FLAG'])
Sentence_Computation = to_date(Sentence_Computation, ["SENTENCE_BEGIN_DATE_(FOR_MAX)","ACTUAL_SENTENCE_END_DATE",\
                                          "PROJECTED_RELEASE_DATE_(PRD)","PAROLE_DISCHARGE_DATE",\
                                          "PAROLE_SUPERVISION_BEGIN_DATE"])
Sentence_Computation = keep_attribute(Sentence_Computation, ["INMATE_DOC_NUMBER","INMATE_COMMITMENT_PREFIX","INMATE_SENTENCE_COMPONENT",\
                                  "SENTENCE_BEGIN_DATE_(FOR_MAX)","ACTUAL_SENTENCE_END_DATE",\
                                  "PROJECTED_RELEASE_DATE_(PRD)","PAROLE_DISCHARGE_DATE",\
                                  "PAROLE_SUPERVISION_BEGIN_DATE","INMATE_COMPUTATION_STATUS_FLAG_ACTIVE",\
                                  "INMATE_COMPUTATION_STATUS_FLAG_EAR.TERM","INMATE_COMPUTATION_STATUS_FLAG_EXPIRED",\
                                  "INMATE_COMPUTATION_STATUS_FLAG_FUTURE","INMATE_COMPUTATION_STATUS_FLAG_PAROLED",\
                                  "INMATE_COMPUTATION_STATUS_FLAG_POST REL"])

new_file_name = file_name[:-4] + '_cleaned.csv'
Sentence_Computation.to_csv(new_file_name)



In [209]:
#Cleaning Disciplinary Infraction Charge
file_name = os.path.join(data_dir, "preprocessed/INMT9CF1.csv")
Infraction_Charge = pd.read_csv(file_name, low_memory=False)
Infraction_Charge = categorical_to_dummy(Infraction_Charge, ['DISCIPLINARY_CHARGE_LEVEL'])
Infraction_Charge = flag_to_dummy(Infraction_Charge, ["ACTIVATE_PRIOR_SUSPENSION"], rename=False)
Infraction_Charge =keep_attribute(Infraction_Charge, ["INMATE_DOC_NUMBER", "DISCIPLINARY_CHARGE_LEVEL_APPEAL",\
                                   "DISCIPLINARY_CHARGE_LEVEL_DISCP HEAR OFFC",\
                                   "DISCIPLINARY_CHARGE_LEVEL_UNIT",'ACTIVATE_PRIOR_SUSPENSION'])
#Shall we include?:
#'DISCIPLINARY_APPEAL_DECISION'

new_file_name = file_name[:-4] + '_cleaned.csv'
Infraction_Charge.to_csv(new_file_name)

In [ ]:
#Cleaning Financial_Obligation

file_name = os.path.join(data_dir, "preprocessed/OFNT1BA1.csv")
Financial_Obligation = pd.read_csv(file_name, low_memory=False)
Financial_Obligation = categorical_to_dummy(Financial_Obligation, ["COURT_ORDERED_PAYMENT_TYPE"])
Financial_Obligation = flag_to_dummy(Financial_Obligation, ["PAYEE_ACCOUNT_STATUS_CODE"], rename=False)
#Financial_Obligation = discretize_variable(Financial_Obligation, ["COP_BALANCE"])
Financial_Obligation = keep_attribute(Financial_Obligation, ["OFFENDER_NC_DOC_ID_NUMBER", "COP_COMMITMENT_PREFIX",\
                                      "COP_ACCOUNT_SEQUENCE_NUMBER","COURT_ORDERED_PAYMENT_TYPE_FINE",\
                                      "COURT_ORDERED_PAYMENT_TYPE_COMM. SERVICE FEE","PAYEE_ACCOUNT_STATUS_CODE",\
                                      "COURT_ORDERED_PAYMENT_TYPE_COURT COSTS","COURT_ORDERED_PAYMENT_TYPE_JAIL FEE",\
                                      "COURT_ORDERED_PAYMENT_TYPE_JUDGEMENT","COURT_ORDERED_PAYMENT_TYPE_RESTITUTION",\
                                      "COURT_ORDERED_PAYMENT_TYPE_SUPERVISION FEE", "COP_BALANCEcat"])

new_file_name = file_name[:-4] + '_cleaned.csv'
Financial_Obligation.to_csv(new_file_name)


In [ ]:
#Cleaning Court Commitment

file_name = os.path.join(data_dir, "preprocessed/OFNT3BB1.csv")
Court_Commitment = pd.read_csv(file_name, low_memory=False, nrows=40000)
Court_Commitment
Court_Commitment = categorical_to_dummy(Court_Commitment, ["COURT_ORDERED_PAYMENT_TYPE"])
Court_Commitment = flag_to_dummy(Court_Commitment, ["PAYEE_ACCOUNT_STATUS_CODE"], rename=False)
#Court_Commitment = discretize_variable(Court_Commitment, ["COP_BALANCE"])
Court_Commitment =  keep_attribute(Court_Commitment, ["OFFENDER_NC_DOC_ID_NUMBER", "COP_COMMITMENT_PREFIX",\
                                      "COP_ACCOUNT_SEQUENCE_NUMBER","COURT_ORDERED_PAYMENT_TYPE_FINE",\
                                      "COURT_ORDERED_PAYMENT_TYPE_COMM. SERVICE FEE","PAYEE_ACCOUNT_STATUS_CODE",\
                                      "COURT_ORDERED_PAYMENT_TYPE_COURT COSTS","COURT_ORDERED_PAYMENT_TYPE_JAIL FEE",\
                                      "COURT_ORDERED_PAYMENT_TYPE_JUDGEMENT","COURT_ORDERED_PAYMENT_TYPE_RESTITUTION",\
                                      "COURT_ORDERED_PAYMENT_TYPE_SUPERVISION FEE", "COP_BALANCEcat"])

new_file_name = file_name[:-4] + '_cleaned.csv'
Court_Commitment.to_csv(new_file_name)



In [224]:
#Cleaning Sentence_Component

file_name =  os.path.join(data_dir, "preprocessed/OFNT3CE1.csv")
Sentence_Component = pd.read_csv(file_name, low_memory=False)#, nrows=40000)

#I don't think we want to filter by FELON:
#Sentence_Component = Sentence_Component[Sentence_Component["PRIMARY_FELONY/MISDEMEANOR_CD."]=="FELON"]
Sentence_Component = to_int(Sentence_Component, ["MAXIMUM_SENTENCE_LENGTH"])
Sentence_Component = categorical_to_dummy(Sentence_Component, ["PRIMARY_FELONY/MISDEMEANOR_CD.",\
                                                               "PUNISHMENT_TYPE_CODE",\
                                                               "COURT_TYPE_CODE",\
                                                               "COMPONENT_DISPOSITION_CODE",\
                                                               "OFFENSE_QUALIFIER_CODE",\
                                                               "INMATE_SENTENCE_STATUS_CODE",\
                                                               "SERVING_MIN_OR_MAX_TERM_CODE"])
Sentence_Component = to_date(Sentence_Component, ["DATE_OFFENSE_COMMITTED_-_BEGIN",\
                                                  "SENTENCE_EFFECTIVE(BEGIN)_DATE",\
                                                  "INMATE_COMPONENT_STATUS_DATE"])
Sentence_Component = keep_attribute(Sentence_Component,['OFFENDER_NC_DOC_ID_NUMBER', 'COMMITMENT_PREFIX',\
                                   'SENTENCE_COMPONENT_NUMBER', 'COUNTY_OF_CONVICTION_CODE',\
                                   'PRIMARY_OFFENSE_CODE', 'DATE_OFFENSE_COMMITTED_-_BEGIN',\
                                   'MINIMUM_SENTENCE_LENGTH', 'MAXIMUM_SENTENCE_LENGTH',\
                                   'LENGTH_OF_SUPERVISION', 'SENTENCE_EFFECTIVE(BEGIN)_DATE',\
                                   'INMATE_COMPONENT_STATUS_DATE', 'PRIMARY_FELONY/MISDEMEANOR_CD._FELON',\
                                   'PRIMARY_FELONY/MISDEMEANOR_CD._MISD.',\
                                   'PRIMARY_FELONY/MISDEMEANOR_CD._UNKN.',\
                                   'PUNISHMENT_TYPE_CODE_ACTIVE  SS',\
                                   'PUNISHMENT_TYPE_CODE_COMMUNITY SS (DCC)', 'PUNISHMENT_TYPE_CODE_DWI',\
                                   'PUNISHMENT_TYPE_CODE_FAIR FELONS',\
                                   'PUNISHMENT_TYPE_CODE_FAIR MISDEMEAN',\
                                   'PUNISHMENT_TYPE_CODE_INTERMEDIATE SS',\
                                   'PUNISHMENT_TYPE_CODE_NON JUDGMENT CASES',\
                                   'PUNISHMENT_TYPE_CODE_NON-N.C. OFF.',\
                                   'PUNISHMENT_TYPE_CODE_POST RELEASE', 'PUNISHMENT_TYPE_CODE_PRE-FAIR',\
                                   'PUNISHMENT_TYPE_CODE_PRE-SS (FAIR) DCC', 'PUNISHMENT_TYPE_CODE_PSI',\
                                   'COURT_TYPE_CODE_CO RECORDR', 'COURT_TYPE_CODE_DISTRICT',\
                                   'COURT_TYPE_CODE_DOMES RELA', 'COURT_TYPE_CODE_ICC/OS/FED',\
                                   'COURT_TYPE_CODE_J.P.', 'COURT_TYPE_CODE_MAGISTRAT',\
                                   'COURT_TYPE_CODE_MAYORS', 'COURT_TYPE_CODE_SUPERIOR',\
                                   'COMPONENT_DISPOSITION_CODE_APPEAL',\
                                   'COMPONENT_DISPOSITION_CODE_BENCH TRIAL',\
                                   'COMPONENT_DISPOSITION_CODE_CRV/3M',\
                                   'COMPONENT_DISPOSITION_CODE_GUILTY',\
                                   'COMPONENT_DISPOSITION_CODE_JUDGMENT',\
                                   'COMPONENT_DISPOSITION_CODE_JURY TRIAL',\
                                   'COMPONENT_DISPOSITION_CODE_NEGOTIATED PLEA',\
                                   'COMPONENT_DISPOSITION_CODE_NOLO CONTENDRE',\
                                   'COMPONENT_DISPOSITION_CODE_NON-JUDGMENT',\
                                   'COMPONENT_DISPOSITION_CODE_NOT GUILTY',\
                                   'COMPONENT_DISPOSITION_CODE_PARTIAL REVOKE',\
                                   'COMPONENT_DISPOSITION_CODE_POST REL. REVOKED',\
                                   'COMPONENT_DISPOSITION_CODE_UNKNOWN',\
                                   'OFFENSE_QUALIFIER_CODE_ACCES A/F', 'OFFENSE_QUALIFIER_CODE_ACCES ATT',\
                                   'OFFENSE_QUALIFIER_CODE_ACCES B/F', 'OFFENSE_QUALIFIER_CODE_AID&ABET',\
                                   'OFFENSE_QUALIFIER_CODE_ATTEMPTED', 'OFFENSE_QUALIFIER_CODE_CONSPI ATT',\
                                   'OFFENSE_QUALIFIER_CODE_CONSPIRACY', 'OFFENSE_QUALIFIER_CODE_PRINCIPAL',\
                                   'OFFENSE_QUALIFIER_CODE_SOLIC ATT', 'OFFENSE_QUALIFIER_CODE_SOLICIT',\
                                   'OFFENSE_QUALIFIER_CODE_UNKNOWN', 'INMATE_SENTENCE_STATUS_CODE_ACTIVE',\
                                   'INMATE_SENTENCE_STATUS_CODE_AMENDED',\
                                   'INMATE_SENTENCE_STATUS_CODE_ARRESTED',\
                                   'INMATE_SENTENCE_STATUS_CODE_CANCEL',\
                                   'INMATE_SENTENCE_STATUS_CODE_COMM T/S',\
                                   'INMATE_SENTENCE_STATUS_CODE_COMMUTAT',\
                                   'INMATE_SENTENCE_STATUS_CODE_CORRECT',\
                                   'INMATE_SENTENCE_STATUS_CODE_COURT OR',\
                                   'INMATE_SENTENCE_STATUS_CODE_DISMISS',\
                                   'INMATE_SENTENCE_STATUS_CODE_MODIFIED',\
                                   'INMATE_SENTENCE_STATUS_CODE_P & T',\
                                   'INMATE_SENTENCE_STATUS_CODE_PAR CONS',\
                                   'INMATE_SENTENCE_STATUS_CODE_PARDON',\
                                   'INMATE_SENTENCE_STATUS_CODE_PC TERM',\
                                   'INMATE_SENTENCE_STATUS_CODE_QUASHED',\
                                   'INMATE_SENTENCE_STATUS_CODE_REINSTATE',\
                                   'INMATE_SENTENCE_STATUS_CODE_RESENTEN',\
                                   'INMATE_SENTENCE_STATUS_CODE_VACATED',\
                                   'SERVING_MIN_OR_MAX_TERM_CODE_MAX.TERM:',\
                                   'SERVING_MIN_OR_MAX_TERM_CODE_MIN.TERM:'])

new_file_name = file_name[:-4] + '_cleaned.csv'
Sentence_Component.to_csv(new_file_name)


In [13]:
#Cleaning Special_Cond_Sanctions

file_name = os.path.join(data_dir, "preprocessed/OFNT3DE1.csv")
Special_Cond = pd.read_csv(file_name, low_memory=False)#, nrows=40000)
Special_Cond = keep_attribute(Special_Cond,["OFFENDER_NC_DOC_ID_NUMBER",\
                                            "COMMITMENT_PREFIX",\
                                            "SENTENCE_COMPONENT_NUMBER",\
                                            "SPECIAL_PROVISION/SANCTION_CD",\
                                            "COMPLETION_STATUS_OF_SANCTION"])

new_file_name = file_name[:-4] + '_cleaned.csv'
Special_Cond.to_csv(new_file_name)

In [17]:
#Cleaning Parole_Analyst

file_name = os.path.join(data_dir, "preprocessed/INMT4CA1.csv")
Parole_Analyst = pd.read_csv(file_name, low_memory=False)#, nrows=40000)
Parole_Analyst = categorical_to_dummy(Parole_Analyst, ['NEXT_PAROLE_REVIEW_TYPE_CODE'])
Parole_Analyst = to_date(Parole_Analyst, ["RELEASE_DATE_(PAROLE_REVIEW)"])
Parole_Analyst = keep_attribute(Parole_Analyst,   ['INMATE_DOC_NUMBER', 'RELEASE_DATE_(PAROLE_REVIEW)',\
                                                   'NEXT_PAROLE_REVIEW_TYPE_CODE_AFTERCARE TRT.',\
                                                   'NEXT_PAROLE_REVIEW_TYPE_CODE_COMMISSION REVW',\
                                                   'NEXT_PAROLE_REVIEW_TYPE_CODE_INTERIM REVIEW',\
                                                   'NEXT_PAROLE_REVIEW_TYPE_CODE_MAPP REVIEW',\
                                                   'NEXT_PAROLE_REVIEW_TYPE_CODE_PAROLE REVIEW',\
                                                   'NEXT_PAROLE_REVIEW_TYPE_CODE_RT DRUG TEST',\
                                                   'NEXT_PAROLE_REVIEW_TYPE_CODE_VOTE REVIEW',\
                                                   'NEXT_PAROLE_REVIEW_TYPE_CODE_WORK RELEASE'])

new_file_name = file_name[:-4] + '_cleaned.csv'
Parole_Analyst.to_csv(new_file_name)


In [ ]:
con = sqlite3.connect(os.path.join(data_dir, "crimes.db"))
cur = con.cursor()

tables = ['INMT4AA1_cleaned', 'INMT4BB1_cleaned', 'INMT9CF1_cleaned',\
          'OFNT1BA1_cleaned', 'OFNT3BB1_cleaned', 'OFNT3CE1_cleaned',\
          'OFNT3DE1_cleaned', 'INMT4CA1_cleaned']

for table in tables:
    #print(table)
    file_name = "data/preprocessed/{}.csv".format(table)
    col_names = pd.read_csv(file_name, nrows=0).columns
    n_columns = len(col_names)
    col_names = clean_str(', '.join(col_names))
    cur.execute('DROP TABLE IF EXISTS {}'.format(table))
    cur.execute("CREATE TABLE {} ({});".format(table, col_names))
    
    #File contains NULL bytes. That's why I replaced '\0' with ''
    reader = csv.reader(x.replace('\0','') for x in open(file_name))
    for row in reader:
        row = [None if x == '' else x for x in row]
        cur.execute("INSERT INTO {} VALUES ({});".format(table,",".join(['?']*n_columns)), row)

con.commit()
con.close()

In [ ]:
#OTHER:
'''
Warrant_Issued = pd.read_csv("data/preprocessed/OFNT9BE1.csv")
Offender_profile = pd.read_csv("data/preprocessed/OFNT3AA1.csv")

Infraction_Charge.groupby("SUSPENSION_STATUS").size()

#Print type of data on the df:

for at in Inmate_Profile.columns:
    print(at)
    print(type(Inmate_Profile[at][0]))
'''